In [ ]:
import numpy as np 
import pandas as pd 

import os 

import matplotlib.pyplot as plt  data

import PIL 
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.keras import layers

image = "../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/person1011_bacteria_2942.jpeg"
PIL.Image.open(image)

training_dir = "../input/chest-xray-pneumonia/chest_xray/train/" 

#Data Augmentation
training_generator = ImageDataGenerator(rescale=1/225, 
                                        featurewise_center=False, 
                                        samplewise_center = False, 
                                        
                                        featurewise_std_normalization=False, 
                                        samplewise_std_normalization = False, 
                                        
                                        zca_whitening = False, 
                                        
                                        rotation_range = 30, 
                                        zoom_range = 0.2, 
                                        width_shift_range = 0.1, 
                                        height_shift_range = 0.1, 
                                        horizontal_flip= False, 
                                        vertical_flip=False)

train_generator = training_generator.flow_from_directory(
training_dir, target_size = (200,200), batch_size = 4, class_mode="binary")

validation_dir = "../input/chest-xray-pneumonia/chest_xray/val/" 

validation_generator = ImageDataGenerator(rescale=1/255)

val_generator = validation_generator.flow_from_directory(
validation_dir,target_size=(200,200),batch_size=4, class_mode="binary" )

test_dir = "../input/chest-xray-pneumonia/chest_xray/test"

test_generator= ImageDataGenerator(rescale=1/255)

test_generator = test_generator.flow_from_directory(test_dir, target_size=(200, 200), batch_size=16, class_mode="binary")

#CNN Model
model = Sequential() 

model.add(layers.Conv2D(32,(3,3), input_shape = (200,200,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3,3,), activation ="relu"))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(256, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.001), loss="binary_crossentropy", metrics=["acc"])

history = model.fit_generator(train_generator, 
                              validation_data = val_generator, 
                              epochs=5, 
                              verbose= 1
                             )


%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

plt.plot(epochs, acc, "r", label="Training Accuracy")
plt.plot(epochs, val_acc, "b", label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend(loc=0)
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.figure()

print("Loss of the model is : ", model.evaluate(test_generator)[0]*100, "%")
print("Accuracy of the model is : " ,model.evaluate(test_generator)[1]*100, "%")